In [27]:
import pandas as pd

df_train = pd.read_csv('train_data_with_features.csv')
df_test = pd.read_csv('test_data_with_features.csv')

# Movemos reason_end al final para el conjunto de train
df_train_cleaned = df_train[[c for c in df_train.columns if c != 'reason_end'] + ['reason_end']]
df_train_cleaned.head()

/var/folders/fx/50zkh6h57t37jktm101z3yvr0000gn/T/ipykernel_93991/61479679.py:3: DtypeWarning: Columns (8,9,10,18,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('train_data_with_features.csv')


,ts,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,...,obs_id,duration_normalized,explicit,popularity,track_number,available_markets_count,artist_uri,album_type,song_age,reason_end
0,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,The Eater Of Dreams,Nine Inch Nails,Hesitation Marks,spotify:track:1IPdwxRUbuNZiRpFN49RQC,NaN,NaN,...,1,0.093570,False,0.0,1.0,0.0,spotify:artist:0X380XXQSNBYuleKzav5UO,album,12.013689,fwdbtn
1,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Copy Of A,Nine Inch Nails,Hesitation Marks,spotify:track:4BFKCEp4gwG3QHNlYodLMy,NaN,NaN,...,2,0.575761,False,0.0,2.0,0.0,spotify:artist:0X380XXQSNBYuleKzav5UO,album,12.013689,unknown
2,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,El extranjero,Bunbury,Singles,spotify:track:7ghPo5JVqcmwZQSpm6uyGG,NaN,NaN,...,3,0.475925,False,20.0,1.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,trackdone
3,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Luna,Bunbury,Singles,spotify:track:4g0Jk6sWxdu3FwwGDtoQPL,NaN,NaN,...,4,0.460320,False,27.0,2.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,fwdbtn
4,2013-10-29 18:40:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Feliz año,Bunbury,Singles,spotify:track:0GZzkyz7MN15llCyCBgPWh,NaN,NaN,...,5,0.430945,False,18.0,3.0,171.0,spotify:artist:4uqzzJg3ww5eH7IgGV7DMT,album,23.912389,fwdbtn


In [28]:
df_train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911344 entries, 0 to 911343
Data columns (total 30 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 911344 non-null  object 
 1   platform                           911344 non-null  object 
 2   conn_country                       911344 non-null  object 
 3   ip_addr                            911344 non-null  object 
 4   master_metadata_track_name         907211 non-null  object 
 5   master_metadata_album_artist_name  907211 non-null  object 
 6   master_metadata_album_album_name   907211 non-null  object 
 7   spotify_track_uri                  907211 non-null  object 
 8   episode_name                       4133 non-null    object 
 9   episode_show_name                  4133 non-null    object 
 10  spotify_episode_uri                4133 non-null    object 
 11  audiobook_title                    0 no

In [29]:
# Limpiamos las columnas nulas
df_train_cleaned = df_train_cleaned.drop(columns=['audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'ip_addr', 
                                        'spotify_track_uri', 'spotify_episode_uri'])
df_test_cleaned = df_test.drop(columns=['audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'ip_addr', 
                                        'spotify_track_uri', 'spotify_episode_uri'])

In [30]:
def clean_and_reorder_timestamp_columns(df):
    df = df.copy()
    df['ts'] = pd.to_datetime(df['ts'])
    df['year'] = df['ts'].dt.year
    df['month'] = df['ts'].dt.month
    df['day'] = df['ts'].dt.day
    df['time_of_day'] = df['ts'].dt.time

    df.pop('ts')
    year = df.pop('year')
    month = df.pop('month')
    day = df.pop('day')
    time_of_day = df.pop('time_of_day')

    df.insert(0, 'year', year)
    df.insert(1, 'month', month)
    df.insert(2, 'day', day)
    df.insert(3, 'time_of_day', time_of_day)
    return df

df_train_cleaned = clean_and_reorder_timestamp_columns(df_train_cleaned)
df_test_cleaned = clean_and_reorder_timestamp_columns(df_test_cleaned)

In [31]:
# Cambiamos la columna reason_end a 1 (fwdbtn) o 0 (otro)
df_train_cleaned['reason_end'] = df_train_cleaned['reason_end'].apply(lambda x: 1 if x == 'fwdbtn' else 0)

In [32]:
df_train_cleaned.to_csv('train_data_cleaned.csv', index=False)
df_test_cleaned.to_csv('test_data_cleaned.csv', index=False)
